In [ ]:
import pandas as pd
import numpy as np
import itertools
import pickle
from collections import Counter
from itertools import combinations, groupby, chain
import warnings
warnings.filterwarnings('ignore')

In [ ]:
clustered_users = pd.read_pickle("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/overall_grouped_users/grouped_users_with_20clusters.pkl")

In [ ]:
aisles = pd.read_csv('/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/aisles.csv')

In [ ]:
products_desc = pd.read_csv('/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/df_products_description.csv')

In [ ]:
print(clustered_users.shape)
clustered_users.head()

(206209, 139)


,num_orders,mode_order_dow,median_order_hour,mean_days_since,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,...,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,cluster
user_id,,,,,,,,,,,,,,,,,,,,,
1,11,4,8.0,19.200001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1
2,15,1,10.0,18.009390,0.0,3.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,42.0,5
3,12,0,16.0,11.487180,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,7
4,5,4,13.0,15.357142,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
5,5,0,16.0,12.314285,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,7


In [ ]:
merged_orders = pd.read_pickle("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/dfmerged_orders_with_products.pkl")

In [ ]:
# I want to calculate the association rules by each cluster.  It will make the data smaller.
merged_orders = merged_orders.sort_values(['order_id','product_id'])
merged_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
18486555,1,112108,train,4,4,10,9.0,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce
18486554,1,112108,train,4,4,10,9.0,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
18486558,1,112108,train,4,4,10,9.0,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce
18486560,1,112108,train,4,4,10,9.0,22035,8,1,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
18486557,1,112108,train,4,4,10,9.0,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods


In [ ]:
clustered_users.reset_index(inplace=True)

In [ ]:
# Merging these to add the cluster to each order
clustered_orders = merged_orders.merge(clustered_users[['user_id','cluster']], on = 'user_id')
print(clustered_orders.shape)
clustered_orders.head()

(33819106, 16)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department,cluster
0,1,112108,train,4,4,10,9.0,10246,3,0,Organic Celery Hearts,83,4,fresh vegetables,produce,7
1,1,112108,train,4,4,10,9.0,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs,7
2,1,112108,train,4,4,10,9.0,13176,6,0,Bag of Organic Bananas,24,4,fresh fruits,produce,7
3,1,112108,train,4,4,10,9.0,22035,8,1,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs,7
4,1,112108,train,4,4,10,9.0,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods,7


In [ ]:
clustered_orders.cluster.value_counts()

5     7536181
14    4580363
7     4290064
1     3398666
19    3312984
6     2110061
16    1594320
13    1555035
4     1538286
17     779876
2      599753
3      498904
18     465213
9      328090
15     302895
0      246011
12     244968
10     207254
8      126541
11     103641
Name: cluster, dtype: int64

# Market Basket Analysis

In [ ]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
'''Python Generators - A generator is a special type of function that returns an iterable sequence of items.
However, unlike regular functions that return all the values at once, a generator yields one value at a time. To get the
next value in the set, we must ask for it — either by explicitly calling the generator’s built-in “next” method, or
implicitly via a for loop.  This is a great property of generators because it means that we don’t have to store all of
the values in memory at once. We can load and process one value at a time, discard that value when we finished, and move
on to process the next value. This feature makes generators perfect for creating item pairs and counting their
frequency of co-occurrence.'''

def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), 
                       left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), 
                       left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [ ]:
association_rules(cluster_orders[0], 0.0001)

Starting number of orders, items, order_item pairs: 29656, 14619, 246011
order_id
191         2178
191        12811
191        24325
191        29116
191        41391
           ...  
3306450    13998
3306450    23288
3306450    26165
3306450    33198
3306450    38444
Name: item_id, Length: 238357, dtype: int32
Items with support >= 0.0001:            8863
Remaning number of orders, items, order_item pairs: 29608, 8863, 238357
Remaining orders with 2+ items:       28009
Remaining number of orders, items, order_item pairs: 28009, 8862, 236758
Item pairs:                          774800
Item pairs with support >= 0.0001:     115467



,item_A,item_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
107993,17982,49566,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
106782,24660,31784,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
90394,29878,43248,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
107254,137,17982,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
50022,4846,37613,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
...,...,...,...,...,...,...,...,...,...,...,...
61804,1463,45190,4,0.000143,341,0.012175,2037,0.072727,0.011730,0.001964,0.161292
82878,45190,47877,3,0.000107,2037,0.072727,276,0.009854,0.001473,0.010870,0.149458
10582,1160,23909,5,0.000179,1107,0.039523,851,0.030383,0.004517,0.005875,0.148659
13515,23909,36425,3,0.000107,851,0.030383,748,0.026706,0.003525,0.004011,0.132004


In [ ]:
def association_rules(order_item, min_support):

#     print("Starting number of order_item pairs: {:22d}".format(len(order_item)))
    print("Starting number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item))) 

    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]
    print(order_item)
    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
#     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaning number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
#     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaining number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                          order_item.nunique(),len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [ ]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id']]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i
    # break

In [ ]:
# run each cluster series through association rule function and obtain association rules at item level

group_association_rules_dic = {}
for x in range(0,20):
    print('Cluster: ', x)
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 29656, 14619, 246011
Items with support >= 0.0001:            8863
Remaning number of orders, items, order_item pairs: 29608, 8863, 238357
Remaining orders with 2+ items:       28009
Remaining number of orders, items, order_item pairs: 28009, 8862, 236758
Item pairs:                          774800
Item pairs with support >= 0.0001:     115467

Cluster:  1
Starting number of orders, items, order_item pairs: 479984, 44452, 3398666
Items with support >= 0.0001:            9241
Remaning number of orders, items, order_item pairs: 474204, 9241, 3028163
Remaining orders with 2+ items:      419257
Remaining number of orders, items, order_item pairs: 419257, 9241, 2973216
Item pairs:                         4809491
Item pairs with support >= 0.0001:      26647

Cluster:  2
Starting number of orders, items, order_item pairs: 43407, 25356, 599753
Items with support >= 0.0001:           13675
Remaning number of orders, items, order_i

In [ ]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
               .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
               .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                    'product_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confAtoB',
                                                                    'confBtoA','lift']]

In [ ]:
pickle.dump(group_association_rules_dic, open("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/mba/group_association_rules_dic.p", "wb"))

In [ ]:
# group_association_rules_dic = pickle.load(open("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/mba/group_association_rules_dic.p", "rb"))

In [ ]:
# example of association rule dataframe

x = group_association_rules_dic[19]
x.sort_values('lift', ascending = False)

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,11451,17207,Peppermint Patties,Non Fat Greek Yogurt,27,0.000107,31,0.000123,42,0.000166,0.870968,0.642857,5234.578341
1,13575,17207,Apples,Non Fat Greek Yogurt,37,0.000147,46,0.000182,42,0.000166,0.804348,0.880952,4834.187888
3,11451,13575,Peppermint Patties,Apples,26,0.000103,31,0.000123,46,0.000182,0.838710,0.565217,4602.382889
8,34519,39618,Simply 100® Blueberry Blended Non-Fat Greek Yo...,Simply 100 Peach Blended Non-Fat Greek Yogurt,30,0.000119,54,0.000214,31,0.000123,0.555556,0.967742,4523.709677
6,17207,41400,Non Fat Greek Yogurt,Crunchy Oats 'n Honey Granola Bars,28,0.000111,42,0.000166,38,0.000151,0.666667,0.736842,4428.473684
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15425,16797,21137,Strawberries,Organic Strawberries,199,0.000788,13331,0.052812,45971,0.182119,0.014928,0.004329,0.081966
4661,5450,47209,Small Hass Avocado,Organic Hass Avocado,54,0.000214,5700,0.022581,32698,0.129537,0.009474,0.001651,0.073135
11307,5876,47626,Organic Lemon,Large Lemon,37,0.000147,10237,0.040555,13880,0.054987,0.003614,0.002666,0.065731
7978,47209,47766,Organic Hass Avocado,Organic Avocado,117,0.000464,32698,0.129537,19795,0.078420,0.003578,0.005911,0.045629


# Market Basket Analysis at Aisle Level

In [ ]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id', 'aisle_id']]
    i = i.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[x] = i
#     break
# cluster_aisle_orders[0]

order_id
191          2
191         28
191         38
191         61
191        124
          ... 
3420688      8
3420688    134
3420732     27
3420732     28
3420732     62
Name: item_id, Length: 185066, dtype: int64

In [ ]:
# run association function at aisle level

group_aisle_association_rules_dic = {}
for x in range(0,20):
    print("Cluster: ", x)
    group_aisle_association_rules_dic[x] = association_rules(cluster_aisle_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 29656, 134, 185066
Items with support >= 0.0001:             134
Remaning number of orders, items, order_item pairs: 29656, 134, 185066
Remaining orders with 2+ items:       27008
Remaining number of orders, items, order_item pairs: 27008, 134, 182418
Item pairs:                            8207
Item pairs with support >= 0.0001:       7400

Cluster:  1
Starting number of orders, items, order_item pairs: 479984, 134, 2559716
Items with support >= 0.0001:             134
Remaning number of orders, items, order_item pairs: 479984, 134, 2559716
Remaining orders with 2+ items:      419255
Remaining number of orders, items, order_item pairs: 419255, 134, 2498987
Item pairs:                            8899
Item pairs with support >= 0.0001:       7285

Cluster:  2
Starting number of orders, items, order_item pairs: 43407, 134, 443492
Items with support >= 0.0001:             134
Remaning number of orders, items, order_item pairs:

In [ ]:
pickle.dump(group_aisle_association_rules_dic, open("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/mba/group_aisle_association_rules_dic.p", "wb"))

In [ ]:
# group_aisle_association_rules_dic = pickle.load(open("/content/drive/MyDrive/Grocery_Recommendation/Kaggle_data/mba/group_aisle_association_rules_dic.p", "rb"))

In [ ]:
# reformat cluster aisle assocation dataframes

for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles
               .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles
               .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                    'aisle_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confAtoB',
                                                                    'confBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})

In [ ]:
x = group_aisle_association_rules_dic[1].sort_values('lift',ascending = False)
x.sort_values('lift',ascending = False)

,aisle_A,aisle_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,28,62,red wines,white wines,1705,0.004067,5227,0.012467,4769,0.011375,0.326191,0.357517,28.676280
45,62,134,white wines,specialty wines champagnes,433,0.001033,4769,0.011375,2040,0.004866,0.090795,0.212255,18.659872
44,28,134,red wines,specialty wines champagnes,447,0.001066,5227,0.012467,2040,0.004866,0.085518,0.219118,17.575315
5634,27,28,beers coolers,red wines,1238,0.002953,6267,0.014948,5227,0.012467,0.197543,0.236847,15.844798
5893,55,73,shave needs,facial care,47,0.000112,977,0.002330,1282,0.003058,0.048106,0.036661,15.732347
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1542,32,74,packaged produce,dish detergents,191,0.000456,39227,0.093564,7904,0.018852,0.004869,0.024165,0.258273
4610,11,32,cold flu allergy,packaged produce,59,0.000141,2730,0.006512,39227,0.093564,0.021612,0.001504,0.230984
571,32,40,packaged produce,dog food care,57,0.000136,39227,0.093564,2675,0.006380,0.001453,0.021308,0.227743
6605,32,58,packaged produce,frozen breads doughs,71,0.000169,39227,0.093564,3426,0.008172,0.001810,0.020724,0.221495


# Recommender

In [ ]:
group_association_rules_dic[0] 

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confAtoB,confBtoA,lift
0,17982,49566,Classic Hummus with Rold Gold Petzels Snack,Peanut Butter Dark Chocolate Fruit & Nut Prote...,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
1,137,49566,Very Vanilla Soymilk,Peanut Butter Dark Chocolate Fruit & Nut Prote...,3,0.000107,3,0.000107,3,0.000107,1.000000,1.000000,9336.333333
2,47175,49566,Blueberry Swirl,Peanut Butter Dark Chocolate Fruit & Nut Prote...,3,0.000107,4,0.000143,3,0.000107,0.750000,1.000000,7002.250000
3,281,49566,Chai Tea Bags,Peanut Butter Dark Chocolate Fruit & Nut Prote...,3,0.000107,4,0.000143,3,0.000107,0.750000,1.000000,7002.250000
4,7788,49566,"Deli Fresh Smoked Ham, 97% Fat Free, Gluten Free",Peanut Butter Dark Chocolate Fruit & Nut Prote...,3,0.000107,4,0.000143,3,0.000107,0.750000,1.000000,7002.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115462,2120,3497,Sauvignon Blanc,"Grape Wine, Delicious Red",7,0.000250,2738,0.097754,52,0.001857,0.002557,0.134615,1.377079
115463,2120,5240,Sauvignon Blanc,Spaghetti No 12,3,0.000107,2738,0.097754,24,0.000857,0.001096,0.125000,1.278716
115464,2120,3876,Sauvignon Blanc,Sofia Blanc De Blancs Sparkling White Wine,4,0.000143,2738,0.097754,36,0.001285,0.001461,0.111111,1.136637
115465,2120,12424,Sauvignon Blanc,Russian River Valley Sauvignon Blanc Wine,4,0.000143,2738,0.097754,38,0.001357,0.001461,0.105263,1.076814


In [ ]:
products_desc.set_index('product_id', inplace=True)

In [ ]:
# returns top x items associated based on lift

def product_to_product(cluster, product_id, item_lift, product_name, num_products):
    df = group_association_rules_dic[cluster]
    df = df[(df['item_A'] == product_id) | (df['item_B'] == product_id)][['product_name_A','item_A','product_name_B','item_B','confAtoB','lift']].sort_values('lift', ascending = False)
    df = df[df['lift'] > item_lift]
    df = df.sort_values('lift', ascending = False)
    df = df.head(n = num_products)
    product_to_product_associations = df['product_name_A'].values.tolist()
    for x in df['product_name_B'].values.tolist():
        product_to_product_associations.append(x)
    product_to_product_associations = [x for x in product_to_product_associations if x != product_name]
    return product_to_product_associations

In [ ]:
# returns recommended products given inputs

def pdp_recommender(user_id, product_id, item_lift, num_products):
    product_name = products_desc.at[product_id,'product_name']
    aisle_id = products_desc.at[product_id,'aisle_id']
    aisle_name = products_desc.at[product_id,'aisle']
    cluster = clustered_users.at[user_id, 'cluster']
    return product_to_product(cluster = cluster, product_id = product_id, item_lift = item_lift, 
                              product_name = product_name, num_products = num_products)

In [ ]:
# results for users in the 20 different clusters on prodcut 39055 Mild Roja Salsa, lift >1, 5 products

for i in range(0,20):
    user = clustered_users[clustered_users['cluster']==i].sample().index[0]
    print('\ncluster', i, ': user ', user)
    print('\n'.join([x for x in (pdp_recommender(user, 39055, 1, 5))]))


cluster 0 : user  28598
Organic Worcestershire
Real Guacamole
Cage Free  100% Liquid Egg Whites
Tortilla Triangles Restaurant Style Chips
Summertime Blues Multigrain Tortilla Chips

cluster 1 : user  169884
Thin & Light Tortilla Chips
Banana
Thick & Crispy Tortilla Chips

cluster 2 : user  3855
Crumbled Cheese, Traditional, Feta, Fat Free
45 Calories & Delight Wheat Bread
Natural Choice Deli Oven Roasted Turkey
Flax Plus Organic Pumpkin Flax Granola
Almond Nut & Rice Cracker Snacks

cluster 3 : user  102062
Thins Light & Tasty Snack Crackers Lightly Salted
Vegan Mac N Cheese
1% Milkfat Low Fat Vitamin A & D Milk
Spinach And Cheese Ravioli
No Salt Added Organic Mixed Vegetables

cluster 4 : user  104409
Mango Tango Fruit Smoothie
Pineapple
Pomelo Grapefruit & Aloe Appeal
Thin & Light Tortilla Chips
Red Mango

cluster 5 : user  194582
Thin & Light Tortilla Chips
Organic Large Brown Grade AA Cage Free Eggs
Original Hummus
Red Peppers
Thick & Crispy Tortilla Chips

cluster 6 : user  19995